## text2mos straight eval

<span style="font-size: 200%; color: red;">First! check the changing "your config.yaml" </span>

## [seq] import

In [1]:
import os
import json
import yaml
import sys
import time
import copy
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm


import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write


import toybox

In [2]:
def plot_audio(audio, samplerate, title='time-domain waveform'):
    """
    usage:
        # audio is [channel, time(num_frames)] ex.torch.Size([1, 68608])
        # audio[0,:]: list of 1ch audio data
        # audio.shape[1]: int value of 1ch audio data length
        audio, sample_rate = torchaudio.load(str(iwav_path))
        %matplotlib inline
        plot_audio(audio, sample_rate)
    """
    # transform to mono
    channel = 0
    audio = audio[channel,:].view(1,-1)
    # to numpy
    audio = audio.to('cpu').detach().numpy().copy()
    time = np.linspace(0., audio.shape[1]/samplerate, audio.shape[1])
    
    fig, ax = plt.subplots(figsize=(12,9))
    
    ax.plot(time, audio[0, :])
    ax.set_title(title, fontsize=20, y=-0.12)
    ax.tick_params(direction='in')
    #ax.set_xlim(0, 3)
    ax.set_xlabel('Time')
    ax.set_ylabel('Amp')
    #ax.legend()
    plt.tight_layout()
    fig.canvas.draw()
    plt.show()
    #fig.savefig('figure.png')
    plt.close(fig)
    return fig

def plot_mel(tensors:list, titles:list[str]):
    """
    usage:
        mel = mel_process(...)
        fig_mel = plot_mel([mel_groundtruth[0], mel_prediction[0]],
                            ['groundtruth', 'inferenced(model)'])

    """
    xlim = max([t.shape[1] for t in tensors])
    fig, axs = plt.subplots(nrows=len(tensors),
                            ncols=1,
                            figsize=(12, 9),
                            constrained_layout=True)

    if len(tensors) == 1:
        axs = [axs]
    
    for i in range(len(tensors)):
        im = axs[i].imshow(tensors[i],
                           aspect="auto",
                           origin="lower",
                           interpolation='none')
        #plt.colorbar(im, ax=axs[i])
        fig.colorbar(im, ax=axs[i])
        axs[i].set_title(titles[i])
        axs[i].set_xlim([0, xlim])
    fig.canvas.draw()
    #plt.show()
    #plt.close()
    plt.close(fig)  # fig.close() 
    return fig

def convert_phn_to_id(phonemes, phn2id):
    """
    phonemes: phonemes separated by ' '
    phn2id: phn2id dict
    """
    return [phn2id[x] for x in ['<bos>'] + phonemes.split(' ') + ['<eos>']]


def text2phnid(text, phn2id, language='en', add_blank=True):
    if language == 'en':
        from text import G2pEn
        word2phn = G2pEn()
        phonemes = word2phn(text)
        if add_blank:
            phonemes = ' <blank> '.join(phonemes)
        return phonemes, convert_phn_to_id(phonemes, phn2id)
    else:
        raise ValueError(
            'Language should be en (for English)!')

In [3]:
# 
# ckpt_file_dir: logs4model/<model_name>/<runtime_name>/ckpt/
#500_397001,500_397003
info_models_e500 = [
    {
        "model_name": "gradtts",
        "config_path": "configs/config_gt_k3.yaml",
        "runtime_name": "run_gt_k3",
        "ckpt_filename": "gradtts_480_381121.pt.pt"
    },
    {
        "model_name": "gradseptts",
        "config_path": "configs/config_sgt_k3.yaml",
        "runtime_name": "run_sgt_k3",
        "ckpt_filename": "gradseptts_480_381121.pt"
    },
    {
        "model_name": "gradtfktts",
        "config_path": "configs/config_tfk_k3.yaml",
        "runtime_name": "run_tfk_k3",
        "ckpt_filename": "gradtfktts_480_381121.pt"
    },
    {
        "model_name": "gradtfk5tts",
        "config_path": "configs/config_tfk_k5.yaml",
        "runtime_name": "run_tfk_k5",
        "ckpt_filename": "gradtfk5tts_480_381121.pt"
    },
    {
        "model_name": "gradtimektts",
        "config_path": "configs/config_timek_k3.yaml",
        "runtime_name": "run_timek_k3",
        "ckpt_filename": "gradtimektts_480_381121.pt"
    },
    {
        "model_name": "gradfreqktts",
        "config_path": "configs/config_freqk_k3.yaml",
        "runtime_name": "run_freqk_k3",
        "ckpt_filename": "gradfreqktts_480_381121.pt"
    },
    {
        "model_name": "gradtfkfultts",
        "config_path": "configs/config_tfkfulmask_k3.yaml",
        "runtime_name": "run_tfkfulmask_k3",
        "ckpt_filename":"gradtfkfulttsmask_500_396308.pt"
    }
]

# logs4model/gradtfkfultts/run_tfkfulmask_k3/ckpt/
"""
    {
        "model_name": "gradtfkfultts",
        "config_path": "configs/config_tfkful_k3.yaml",
        "runtime_name": "run_tfkful_k3",
        "ckpt_filename": "gradtfkfultts_480_381123.pt"
    }
]
"""

info_models_e1000 = [
    {
        "model_name": "gradtts",
        "config_path": "configs/config_gt_k3.yaml",
        "runtime_name": "run_gt_k3",
        "ckpt_filename": "gradtts_1000_794002.pt"
    },
    {
        "model_name": "gradseptts",
        "config_path": "configs/config_sgt_k3.yaml",
        "runtime_name": "run_sgt_k3",
        "ckpt_filename": "gradseptts_1000_794002.pt"
    },
    {
        "model_name": "gradtfktts",
        "config_path": "configs/config_tfk_k3.yaml",
        "runtime_name": "run_tfk_k3",
        "ckpt_filename": "gradtfktts_1000_794002.pt"
    },
    {
        "model_name": "gradtfk5tts",
        "config_path": "configs/config_tfk_k5.yaml",
        "runtime_name": "run_tfk_k5",
        "ckpt_filename": "gradtfk5tts_1000_794002.pt"
    },
    {
        "model_name": "gradtimektts",
        "config_path": "configs/config_timek_k3.yaml",
        "runtime_name": "run_timek_k3",
        "ckpt_filename": "gradtimektts_1000_794002.pt"
    },
    {
        "model_name": "gradfreqktts",
        "config_path": "configs/config_freqk_k3.yaml",
        "runtime_name": "run_freqk_k3",
        "ckpt_filename": "gradfreqktts_1000_794002.pt"
    }
]



target_epoch = 500
print(f'target_epoch: {target_epoch}')
if target_epoch == 500:
    info_models = copy.deepcopy(info_models_e500)
elif target_epoch == 1000:
    info_models = copy.deepcopy(info_models_e1000)
else:
    print('Do not supported')

target_epoch: 500


you can choose

- <span style="font-size: 200%; color: red;">inference section</span>
- <span style="font-size: 200%; color: red;">json analysis section</span>

if you choose <span style="color: red;">inference section</span>,
you keep go ahead below process.

but you choose <span style="color: red;">json analysis section</span>,
you need to <span style="color: blue;">jump to the end this page</span>.

## [seq] check configuration

In [6]:
# First, please check changing <model_name>
config_yaml = 'configs/config_exp_mid.yaml'
config = toybox.load_yaml_and_expand_var('configs/config_exp_mid.yaml')

In [7]:
# for model
model_name = config['model_name']
if model_name == 'gradtts':
    choise_idx = 0
elif model_name == 'gradseptts':
    choise_idx = 1
elif model_name == 'gradtfktts':
    choise_idx = 2
elif model_name == 'gradtfk5tts':
    choise_idx = 3
elif model_name == 'gradtimektts':
    choise_idx = 4
elif model_name == 'gradfreqktts':
    choise_idx = 5
elif model_name == 'gradtfkfultts':
    choise_idx = 6
else:
    os._exit(os.EX_OK)
    print('Alart: carefully chose model ')
print(f'model_name: {model_name}')

# for runtime to load model
runtime_name = config['runtime_name']
print(f'runtime_name: {runtime_name}')
config_path4model = info_models[choise_idx]["config_path"]
runtime_name4model = info_models[choise_idx]["runtime_name"]
ckpt_dir = f'logs4model/{model_name}/{runtime_name4model}/ckpt'
ckpt_path = ckpt_dir + "/" + info_models[choise_idx]["ckpt_filename"]
print(f"ckpt_path: {ckpt_path}")

model_name: gradtfkfultts
runtime_name: run_tfkfulmask_k3
ckpt_path: logs4model/gradtfkfultts/run_tfkfulmask_k3/ckpt/gradtfkfulttsmask_500_396308.pt


In [8]:
# for audio params
n_mels: int = config['n_mels'] # 80
n_fft: int = config['n_fft'] # 1024
sample_rate: int = config['sample_rate'] # 22050
hop_size: int = config['hop_size'] # 256
win_size: int = config['win_size'] # 1024
f_min: int = config['f_min'] # 0
f_max: int = config['f_max'] # 8000
random_seed: int = config['random_seed'] # 1234
print(n_mels, n_fft, sample_rate, hop_size, win_size, f_min, f_max, random_seed)

80 1024 22050 256 1024 0 8000 1234


In [9]:
print(f"phn2id_path: {config['phn2id_path']}")
with open(config['phn2id_path']) as f:
    phn2id = json.load(f)

vocab_size = len(phn2id) + 1

phn2id_path: ./configs/phn2id.json


In [10]:
# for hifigan
# setting file paths
# from https://github.com/huawei-noah/Speech-Backbones/tree/main/Grad-TTS/hifi-gan
# https://drive.google.com/drive/folders/1-eEYTB5Av9jNql0WGBlRoi-WH2J7bp5Y?usp=sharing
HiFiGAN_CONFIG = './hifigan/official_pretrained/LJ_V2/config.json'
HiFiGAN_ckpt = './hifigan/official_pretrained/LJ_V2/generator_v2'

from hifigan import models, env

with open(HiFiGAN_CONFIG) as f:
    hifigan_hparams = env.AttrDict(json.load(f))

hifigan_randomseed = hifigan_hparams.seed
print(f'hifigan_randomseed: {hifigan_randomseed}')

hifigan_randomseed: 1234


## [seq] device setting

In [11]:
import os

print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.
print(f"GPU_name: {torch.cuda.get_device_name()}\nGPU avail: {torch.cuda.is_available()}\n")

all cpu at using device: 52
Number of available CPU: 4


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [12]:
DEVICE = 'cpu' # 'cuda' or 'cpu'
flag_avail_cuda = torch.cuda.is_available()
if flag_avail_cuda == True:
    print('avail cuda')
    print('use cuda')
elif flag_avail_cuda == False and DEVICE=='cpu':
    print('Not avail cuda')
    print('use cpu')
else:
    os._exit(os.EX_OK)

device = torch.device(DEVICE)
print(f'device: {device}')

# setting random_seed ==============
print(f'device: {random_seed}')
toybox.set_seed(random_seed)

# reprint target epoch for ckpt
print(f'check for target_epoch: {target_epoch}')

Not avail cuda
use cpu
device: cpu
device: 1234
check for target_epoch: 500


In [13]:
print(str(torch.get_default_device()))

cpu


## [seq] setting path

In [14]:
# for test_dataset
# RESULT_DIR_PATH = RESULT_DIR_PATH
# RESULT_MEL_DIR_PATH = IMEL_DIR_PATH
# RESULT_WAV_DIR_PATH = IWAV_DIR_PATH
# RESULT_JSON_DIR_PATH = RESULT_JSON_PATH
test_ds_path = Path(config['test_datalist_path'])

# text2mel
#RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}')
#RESULT_MEL_DIR_PATH = RESULT_DIR_PATH / 'mel'
#print(RESULT_MEL_DIR_PATH)

# mel2wav
#IMEL_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/mel')
#RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}')
#RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / 'wav'

# for utmos
#IWAV_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/wav')
#RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}')
#RESULT_JSON_DIR_PATH = RESULT_DIR_PATH / RESULTS_JSON_NAME

if model_name == 'groundtruth':
    # for mel2wav
    #IMEL_DIR_PATH = Path('./result4eval/infer4PBL/groundtruth/cuda/mel/')
    RESULT_MEL_DIR_PATH = Path('./result4eval/infer4PBL/groundtruth/cuda/mel/')
    # for utmos
    #/result4eval/infer4PBL/groundtruth/cuda/
    #IWAV_DIR_PATH = Path('./result4eval/infer4PBL/groundtruth/cuda/wav/')
    RESULT_WAV_DIR_PATH = Path('./result4eval/infer4PBL/groundtruth/cuda/wav/')
    RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}')

# style for mid
#RESULTS_JSON_NAME = 'eval4mid.json'
#RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}')
#RESULT_MEL_DIR_PATH = RESULT_DIR_PATH / 'mel'
#RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / 'wav'
#RESULT_JSON_PATH = RESULT_DIR_PATH / RESULTS_JSON_NAME

RESULTS_JSON_NAME = 'eval4mid.json'
RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/{"e"+str(target_epoch)}')
RESULT_MEL_DIR_PATH = RESULT_DIR_PATH / 'mel'
RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / 'wav'
RESULT_JSON_PATH = RESULT_DIR_PATH / RESULTS_JSON_NAME
RESULTS_JSON_NAME = 'eval4mid.json'
print(f'RESULT_DIR_PATH: {RESULT_DIR_PATH}')

RESULT_DIR_PATH: result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500


In [16]:
# check path

# for text2mel
print('test_ds_path-----------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(config['test_datalist_path']) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

print('RESULT_DIR_PATH-------------------------------------------')
if RESULT_DIR_PATH.exists():
    print(f'Exists {RESULT_DIR_PATH}')
else:
    RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_DIR_PATH}')

print('RESULT_MEL_DIR_PATH-------------------------------------------')
if RESULT_MEL_DIR_PATH.exists():
    print(f'Exists {RESULT_MEL_DIR_PATH}')
else:
    RESULT_MEL_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_MEL_DIR_PATH}')

print('RESULT_WAV_DIR_PATH-------------------------------------------')
if RESULT_WAV_DIR_PATH.exists():
    print(f'Exists {RESULT_WAV_DIR_PATH}')
else:
    RESULT_WAV_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_WAV_DIR_PATH}')

print('RESULT_JSON_PATH-------------------------------------------')
if RESULT_JSON_PATH.exists():
    print(f'Exists {RESULT_JSON_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_JSON_PATH}')

test_ds_path-----------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
RESULT_DIR_PATH-------------------------------------------
Exists result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500
RESULT_MEL_DIR_PATH-------------------------------------------
Exists result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/mel
RESULT_WAV_DIR_PATH-------------------------------------------
Exists result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/wav
RESULT_JSON_PATH-------------------------------------------
No exist result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/eval4mid.json


## [seq] load Model

In [17]:
# import models
from gradtts import GradTTS
from gradseptts import GradSepTTS
from gradtfktts import GradTFKTTS
from gradtfk5tts import GradTFKTTS as GradTFK5TTS
from gradtimektts import GradTimeKTTS
from gradfreqktts import GradFreqKTTS
from gradtfkfultts import GradTFKFULTTS

print(model_name)
print("[seq] loading Model")

print("loading diffusion-TTS ===================================")
N_STEP = 50
TEMP = 1.5

print('loading ', ckpt_path)
_, _, state_dict = torch.load(ckpt_path,
                            map_location=device)

#with open(config_path4model) as f:
#    config = yaml.load(f, yaml.SafeLoader)
config4model = toybox.load_yaml_and_expand_var(config_path4model)

print("[seq] Initializing diffusion-TTS...")
if model_name == "gradtts":
    model = GradTTS.build_model(config4model, vocab_size)
elif model_name == "gradseptts":
    model = GradSepTTS.build_model(config4model, vocab_size)
elif model_name == "gradtfktts":
    model = GradTFKTTS.build_model(config4model, vocab_size)
elif model_name == "gradtfk5tts":
    model = GradTFK5TTS.build_model(config4model, vocab_size)
elif model_name == "gradtfkfultts":
    model = GradTFKFULTTS.build_model(config4model, vocab_size)
elif model_name == "gradtimektts":
    model = GradTimeKTTS.build_model(config4model, vocab_size)
elif model_name == "gradfreqktts":
    model = GradFreqKTTS.build_model(config4model, vocab_size)
else:
    raise ValueError(f"Error: '{model_name}' is not supported")

model = model.to(device)
model.load_state_dict(state_dict)
print(f'Number of encoder + duration predictor parameters: {model.encoder.nparams/1e6}m')
print(f'Number of decoder parameters: {model.decoder.nparams/1e6}m')
print(f'Total parameters: {model.nparams/1e6}m')

print("loading HiFi-GAN ===================================")
"""
#setting file paths
# from https://github.com/huawei-noah/Speech-Backbones/tree/main/Grad-TTS/hifi-gan
# https://drive.google.com/drive/folders/1-eEYTB5Av9jNql0WGBlRoi-WH2J7bp5Y?usp=sharing
HiFiGAN_CONFIG = './hifigan/official_pretrained/LJ_V2/config.json'
HiFiGAN_ckpt = './hifigan/official_pretrained/LJ_V2/generator_v2'

from hifigan import models, env

with open(HiFiGAN_CONFIG) as f:
    hifigan_hparams = env.AttrDict(json.load(f))

hifigan_randomseed = hifigan_hparams.seed
print(f'hifigan_randomseed: {hifigan_randomseed}')
"""
# generator ===================
print("[seq] loading HiFiGAN")
vocoder = models.Generator(hifigan_hparams)

vocoder.load_state_dict(torch.load(
    HiFiGAN_ckpt, map_location=device)['generator'])
vocoder = vocoder.eval().to(device)
vocoder.remove_weight_norm()

print("loading UTMOS ===================================")
predictor_utmos = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)



gradtfkfultts
[seq] loading Model
loading diffusion-TTS ===================================
loading  logs4model/gradtfkfultts/run_tfkfulmask_k3/ckpt/gradtfkfulttsmask_500_396308.pt
[seq] Initializing diffusion-TTS...
Number of encoder + duration predictor parameters: 3.549137m
Number of decoder parameters: 2.056663m
Total parameters: 5.6058m
loading HiFi-GAN ===================================
[seq] loading HiFiGAN


/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...
loading UTMOS ===================================


Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0


In [18]:
"""
ttext = test_ds_list[0]['text']
tphonemes, tphnid = text2phnid(ttext, phn2id, 'en')
print(tphonemes)
print(len(tphonemes))
print(len(tphnid))

tphnid_len = torch.tensor(len(tphnid), dtype=torch.long).unsqueeze(0).to(device)
tphnid = torch.tensor(tphnid).unsqueeze(0).to(device)
print(tphnid_len)
print(tphnid)
"""

"\nttext = test_ds_list[0]['text']\ntphonemes, tphnid = text2phnid(ttext, phn2id, 'en')\nprint(tphonemes)\nprint(len(tphonemes))\nprint(len(tphnid))\n\ntphnid_len = torch.tensor(len(tphnid), dtype=torch.long).unsqueeze(0).to(device)\ntphnid = torch.tensor(tphnid).unsqueeze(0).to(device)\nprint(tphnid_len)\nprint(tphnid)\n"

## [seq] infer

In [19]:
infer_data_num: int = 101 #len(test_ds_list) is 200
print(f'infer_data_num: {infer_data_num}')
print(f'RESULT_DIR_PATH: {RESULT_DIR_PATH}')
print(f'RESULT_MEL_DIR_PATH: {RESULT_MEL_DIR_PATH}')
print(f'RESULT_WAV_DIR_PATH: {RESULT_WAV_DIR_PATH}')
print(f'RESULT_JSON_PATH: {RESULT_JSON_PATH}')

infer_data_num: 101
RESULT_DIR_PATH: result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500
RESULT_MEL_DIR_PATH: result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/mel
RESULT_WAV_DIR_PATH: result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/wav
RESULT_JSON_PATH: result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/eval4mid.json


In [20]:
#a_num = range(90,101)
#print(range(infer_data_num)[-1])
#print(a_num[-1])

In [21]:
eval_list = []


for i in tqdm(range(infer_data_num)):
    test_ds_filename = test_ds_list[i]['name']
    mel_npy_path = RESULT_MEL_DIR_PATH / f"{test_ds_filename}.npy"
    synth_wav_path = RESULT_WAV_DIR_PATH / f"{test_ds_filename}.wav"
    print(f'test_ds_index_{i}: {test_ds_filename}')
    # [seq]text2mel =========================================================
    # load txt
    print('[seq]text2mel')
    text = test_ds_list[i]['text']
    phonemes, phnid = text2phnid(text, phn2id, 'en')
    phonemes_len_int = len(phonemes)
    phnid_len_int = len(phnid)
    print(f'phonemes_len: {phonemes_len_int}')
    print(f'phnid_len: {phnid_len_int}')
    phnid_len = torch.tensor(len(phnid), dtype=torch.long).unsqueeze(0).to(device)
    phnid = torch.tensor(phnid).unsqueeze(0).to(device)

    # [seq] synth speech
    # process text to mel
    # mel is [n_mels, n_frame]
    start_time = time.perf_counter()
    _, mel_prediction, _ = model.forward(phnid,
                                        phnid_len,
                                        n_timesteps=N_STEP,
                                        temperature=TEMP,
                                        solver='original')
    end_time = time.perf_counter()

    dt = end_time - start_time
    dt4mel = dt * 22050 / ( mel_prediction.shape[-1] * 256)
    print(f'{model_name} dt: {dt}')
    print(f'{model_name} RTF: {dt4mel}')
    
    # for save mel
    mel4save = mel_prediction.unsqueeze(0) # [batch, channel(freq), n_frame(time)] ex.[1, 80, 619]
    # save
    #mel_npy_path =  RESULT_MEL_DIR_PATH / f"{test_ds_filename}.npy"
    #print(f'test_ds_index_{i}: {mel_npy_path}')
    np.save(mel_npy_path, mel4save.cpu().detach().numpy().copy())

    # [seq]mel2wav =========================================================
    print('[seq]mel2wav')
    x = np.load(mel_npy_path) # [1, n_mel, n_frame]
    x2audio = torch.FloatTensor(x).to(device)
    x2audio = x2audio.squeeze().unsqueeze(0)
    # x2audio is [1, n_mels, n_frames]
    assert x2audio.shape[0] == 1
    with torch.no_grad():
        # vocoder.forward(x).cpu() is torch.Size([1, 1, 167168])
        audio = (vocoder.forward(x2audio).cpu().squeeze().clamp(-1,1).numpy() * 32768).astype(np.int16)
    write(
        synth_wav_path,
        hifigan_hparams.sampling_rate,
        audio)

    # [seq]wav2utmos =========================================================
    print('[seq]wav2utmos')
    #iwav_path = RESULT_WAV_DIR_PATH / f"{filename}.wav"
    #wav, samplerate = torchaudio.load(iwav_path)
    wav, samplerate = torchaudio.load(synth_wav_path)
    score_utmos = predictor_utmos(wav, samplerate)
    score_utmos_float = score_utmos.item()
    print(f'utmos: {score_utmos_float}')
    #eval_dict = {'name': filename, 'path': str(iwav_path), 'utmos': score_float}
    #score_utmos_list.append(eval_dict)
    
    # path, テキスト文、phonimes, phonimes数, dt, RTF, utmos
    eval_dict = {
        'name': test_ds_filename,
        'phonemes_len': phonemes_len_int,
        'phnid_len': phnid_len_int,
        'dt': dt,
        'RTF4mel': dt4mel,
        'utmos': score_utmos_float
    }
    eval_list.append(eval_dict)
    

  0%|                                                                                                     | 0/101 [00:00<?, ?it/s]

test_ds_index_0: LJ045-0049
[seq]text2mel
phonemes_len: 1127
phnid_len: 215
gradtfkfultts dt: 22.235576699487865
gradtfkfultts RTF: 2.919531644338959
[seq]mel2wav
[seq]wav2utmos


  1%|▉                                                                                          | 1/101 [00:42<1:10:08, 42.08s/it]

utmos: 3.0999863147735596
test_ds_index_1: LJ017-0027
[seq]text2mel
phonemes_len: 411
phnid_len: 79
gradtfkfultts dt: 7.859424853697419
gradtfkfultts RTF: 2.5739709782561206
[seq]mel2wav
[seq]wav2utmos


  2%|█▊                                                                                           | 2/101 [00:52<39:03, 23.67s/it]

...

utmos: 3.257174491882324
test_ds_index_99: LJ029-0081
[seq]text2mel
phonemes_len: 1427
phnid_len: 269
gradtfkfultts dt: 27.369109156541526
gradtfkfultts RTF: 2.39814684361386
[seq]mel2wav
[seq]wav2utmos


 99%|██████████████████████████████████████████████████████████████████████████████████████████ | 100/101 [32:14<00:26, 26.54s/it]

utmos: 3.9679741859436035
test_ds_index_100: LJ017-0230
[seq]text2mel
phonemes_len: 1085
phnid_len: 207
gradtfkfultts dt: 17.960323691368103
gradtfkfultts RTF: 2.184990385519656
[seq]mel2wav
[seq]wav2utmos


100%|███████████████████████████████████████████████████████████████████████████████████████████| 101/101 [32:38<00:00, 19.39s/it]

utmos: 3.4414823055267334


In [22]:
#RESULT_JSON_PATH = RESULT_DIR_PATH / 'eval4mid.json'
if RESULT_JSON_PATH.exists() == False:
    with open(RESULT_JSON_PATH, 'w') as f:
        for entry in eval_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {RESULT_JSON_PATH}')
else:
    print(f'Already Exists {RESULT_JSON_PATH}')

Make result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/eval4mid.json


In [23]:
eval_list

[{'name': 'LJ045-0049',
  'phonemes_len': 1127,
  'phnid_len': 215,
  'dt': 22.235576699487865,
  'RTF4mel': 2.919531644338959,
  'utmos': 3.0999863147735596},
 {'name': 'LJ017-0027',
  'phonemes_len': 411,
  'phnid_len': 79,
  'dt': 7.859424853697419,
  'RTF4mel': 2.5739709782561206,
  'utmos': 3.902704954147339},
 {'name': 'LJ023-0031',
  'phonemes_len': 223,
  'phnid_len': 45,
  'dt': 4.456846786662936,
  'RTF4mel': 2.7033855536399027,
  'utmos': 3.1458921432495117},
 {'name': 'LJ032-0046',
  'phonemes_len': 650,
  'phnid_len': 125,
  'dt': 11.059734380804002,
  'RTF4mel': 2.075394395907614,
  'utmos': 3.5897765159606934},
 {'name': 'LJ030-0026',
  'phonemes_len': 1025,
  'phnid_len': 195,
  'dt': 15.682630959898233,
  'RTF4mel': 2.225352738015831,
  'utmos': 2.734996795654297},
 {'name': 'LJ020-0041',
  'phonemes_len': 1154,
  'phnid_len': 219,
  'dt': 20.763847924768925,
  'RTF4mel': 2.334789321256705,
  'utmos': 3.271646022796631},
 {'name': 'LJ001-0070',
  'phonemes_len': 1137,


## recheck eval_json

In [60]:
target_model = 'gradtfk5tts'
target_device = 'cpu'
target_epoch_str = 'e500'

print(target_model)
print(RESULT_DIR_PATH)

# /result4eval/infer4mid/gradtts/cpu/e500/eval4mid.json
# result4eval/<runtime_name>/<model_name>/<device_name>/<target_epoch>/eval.json
# or
# result4eval/json4mid/<target_epoch>/eval4mid_<model_shortword>_<kernel_size>.json
"""
eval_info = {
    'gradtts_cpu': 'result4eval/infer4mid/gradtts/cpu/eval4mid.json',
    'gradseptts_cpu': 'result4eval/infer4mid/gradseptts/cpu/eval4mid.json',
    'gradtfktts_cpu': 'result4eval/infer4mid/gradtfktts/cpu/eval4mid.json',
    'gradtfk5tts_cpu': 'result4eval/infer4mid/gradtfk5tts/cpu/eval4mid.json',
    'gradtimektts_cpu': 'result4eval/infer4mid/gradtimektts/cpu/eval4mid.json',
    'gradfreqktts_cpu': 'result4eval/infer4mid/gradfreqktts/cpu/eval4mid.json',
}
"""
"""
eval_info = {
    'cpu': {
        'e500': {
            'gradtts':f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradseptts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradtfktts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradtfk5tts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradtimektts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradfreqktts': f'{RESULT_DIR_PATH}/eval4mid.json',
        },
        'e1000':{
            'gradtts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradseptts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradtfktts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradtfk5tts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradtimektts': f'{RESULT_DIR_PATH}/eval4mid.json',
            'gradfreqktts': f'{RESULT_DIR_PATH}/eval4mid.json',
        }
    }
}
"""

eval_jsonl_path = RESULT_DIR_PATH / 'eval4mid.json'
print(f'eval_jsonl_path: {eval_jsonl_path}')

if eval_jsonl_path.exists() == True:
    print(f'Exist {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists {eval_jsonl_path}')

gradtfk5tts
result4eval/run_tfk_k5/gradtfk5tts/cpu/e500
eval_jsonl_path: result4eval/run_tfk_k5/gradtfk5tts/cpu/e500/eval4mid.json
Exist result4eval/run_tfk_k5/gradtfk5tts/cpu/e500/eval4mid.json


In [61]:
eval_jsonl_list

[{'name': 'LJ045-0049',
  'phonemes_len': 1127,
  'phnid_len': 215,
  'dt': 18.630400406196713,
  'RTF4mel': 2.1684983580903587,
  'utmos': 3.7940917015075684},
 {'name': 'LJ017-0027',
  'phonemes_len': 411,
  'phnid_len': 79,
  'dt': 5.441780355758965,
  'RTF4mel': 1.8526318065168783,
  'utmos': 4.25128173828125},
 {'name': 'LJ023-0031',
  'phonemes_len': 223,
  'phnid_len': 45,
  'dt': 4.267865874804556,
  'RTF4mel': 1.9657930009074296,
  'utmos': 4.103524684906006},
 {'name': 'LJ032-0046',
  'phonemes_len': 650,
  'phnid_len': 125,
  'dt': 8.511940375901759,
  'RTF4mel': 1.6549827639023154,
  'utmos': 3.930527687072754},
 {'name': 'LJ030-0026',
  'phonemes_len': 1025,
  'phnid_len': 195,
  'dt': 12.719644932076335,
  'RTF4mel': 1.8824377869434814,
  'utmos': 3.6048166751861572},
 {'name': 'LJ020-0041',
  'phonemes_len': 1154,
  'phnid_len': 219,
  'dt': 15.45363965909928,
  'RTF4mel': 1.7491004562414747,
  'utmos': 3.749933958053589},
 {'name': 'LJ001-0070',
  'phonemes_len': 1137,


In [7]:
#'result4eval/run_tfkful_k3/gradtfkfultts/cpu/e500/eval4mid.json'
#result4eval/run_tfkful_k3/gradtfkfultts_e500/cpu/e500/eval4mid.json
#'result4eval/run_tfk_k5/gradtfk5tts/cpu/e500/eval4mid.json
#toybox.load_json('result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4midb.json')
a = toybox.load_json('result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/eval4mid.json')
#a = eval_list

Exist result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/eval4mid.json


In [8]:
dt_list = [a[n]['dt'] for n in range(len(a))]
RTF4mel_list = [a[n]['RTF4mel'] for n in range(len(a))]
utmos_list = [a[n]['utmos'] for n in range(len(a))]

dt_nparr = np.array(dt_list[1:101])
RTF4mel_nparr = np.array(RTF4mel_list[1:101])
utmos_nparr = np.array(utmos_list[1:101])
print(len(dt_nparr))

significant_digits = 10

# for culc difference time to infer text2mel
dt_mean = toybox.round_significant_digits(np.mean(dt_nparr), significant_digits=significant_digits)
dt_var = toybox.round_significant_digits(np.var(dt_nparr), significant_digits=significant_digits)
dt_std = toybox.round_significant_digits(np.std(dt_nparr), significant_digits=significant_digits)
print(f'dt ---------------------------')
print(f'dt mean: {dt_mean}')
print(f'dt var: {dt_var}')
print(f'dt std: {dt_std}')

# for culc RTF4mel to infer text2mel
RTF4mel_mean = toybox.round_significant_digits(np.mean(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_var = toybox.round_significant_digits(np.var(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_std = toybox.round_significant_digits(np.std(RTF4mel_nparr), significant_digits=significant_digits)
print(f'RTF ---------------------------')
print(f'RTF mean: {RTF4mel_mean}')
print(f'RTF var: {RTF4mel_var}')
print(f'RTF std: {RTF4mel_std}')

# for culc utmos to infer 
print(f'utmos ---------------------------')
utmos_mean = toybox.round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = toybox.round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = toybox.round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

100
dt ---------------------------
dt mean: 14.75482539
dt var: 29.438431780000002
dt std: 5.425719471
RTF ---------------------------
RTF mean: 2.221159653
RTF var: 0.02777350155
RTF std: 0.1666538375
utmos ---------------------------
utmos mean: 3.39501344
utmos var: 0.1545336726
utmos std: 0.3931077112


## [seq] analysis json

It is sequence of <span style="color: red;">json analysis section</span> from below cells

The 0th data seemed to have a slower inference time than usual, perhaps because the model took longer to load. Therefore, we try to use data from 1 to 101.

In [23]:
# for loading eval_jsonl of each models
target_model = 'gradtfkfultts'
target_device = 'cpu'
target_epoch_str = 'e500'
eval_json_dir = 'result'#'result4eval/json4mid' # or result4eval/infer4mid

eval_json_dirpath = f'{eval_json_dir}/{target_epoch_str}'
print(f'target_model: {target_model}')
print(f'target_jsonl_path: {eval_json_dirpath}')


# original eval4mid.json path: 'gradtts_cpu': 'result4eval/infer4mid/gradtts/cpu/eval4mid.json'
# diff result4eval/infer4mid/gradtts/cpu/eval4mid.json result4eval/json4mid/eval4mid_gt_k3.json
# no difference between upper files

"""midtermstyle
eval_info = {
    'lj': f'result4eval/infer4PBL/groundtruth/evalljPBL.json',
    'lj_via_hifigan': f'result4eval/infer4PBL/groundtruth/eval4PBL.json',
    'gradtts_cpu': f'{eval_json_dirpath}/eval4mid_gt_k3.json',
    'gradseptts_cpu': f'{eval_json_dirpath}/eval4mid_sgt_k3.json',
    'gradtfktts_cpu': f'{eval_json_dirpath}/eval4mid_tfk_k3.json',
    'gradtfk5tts_cpu': f'{eval_json_dirpath}/eval4mid_tfk_k5.json',
    'gradtimektts_cpu': f'{eval_json_dirpath}/eval4mid_timek_k3.json',
    'gradfreqktts_cpu': f'{eval_json_dirpath}/eval4mid_freqk_k3.json',
}

"""

# result4eval/<runtime_name>/<model_name>/<device_name>/<target_epoch>/eval.json
# or
# result4eval/json4mid/<target_epoch>/eval4mid_<model_shortword>_<kernel_size>.json

eval_info = {
    'lj': f'result4eval/infer4PBL/groundtruth/evalljPBL.json',
    'lj_via_hifigan': f'result4eval/infer4PBL/groundtruth/eval4PBL.json',
    'cpu': {
        'e500': {
            'gradtts':f'{eval_json_dirpath}/eval4mid_gt_k3.json',
            'gradseptts': f'{eval_json_dirpath}/eval4mid_sgt_k3.json',
            'gradtfktts': f'{eval_json_dirpath}/eval4mid_tfk_k3.json',
            'gradtfk5tts': f'{eval_json_dirpath}/eval4mid_tfk_k5.json',
            'gradtimektts': f'{eval_json_dirpath}/eval4mid_timek_k3.json',
            'gradfreqktts': f'{eval_json_dirpath}/eval4mid_freqk_k3.json',
        },
        'e1000':{
            
        }
    }
}

#eval_jsonl_path = Path(eval_info[eval_target]) # midtermstyle
eval_jsonl_path = Path(eval_info[target_device][target_epoch_str][target_model])


if eval_jsonl_path.exists() == True:
    print(f'Exist: {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists: {eval_jsonl_path}')


# for loading test_dataset.json
test_dataset_json_path = Path('configs/test_dataset.json')
import json
with open(test_dataset_json_path) as f:
    test_dataset_list = [json.loads(l) for l in f]
print(f'load: {test_dataset_json_path}')

target_model: gradtts
target_jsonl_path: result4eval/json4mid/e500
Exist: result4eval/json4mid/e500/eval4mid_gt_k3.json
load: configs/test_dataset.json


In [24]:
def round_significant_digits(value, significant_digits=5):
    if value == 0:
        return 0

    import math
    scale = math.floor(-math.log10(abs(value)))  # Find the first nonzero after the decimal point
    factor = 10 ** (scale + significant_digits - 1)  # Scale to hold 5 significant digits

    rounded_value = round(value * factor,1) / factor  # Adjust and round off the scale
    return rounded_value

#input_value = 0.06238165879957992
#input_value = 0.007710418871435095

# Rounded to the nearest whole number to ensure significant figures
#result = round_significant_digits(input_value, significant_digits=5)
#print(result) 

In [25]:
# for culc phonemes length
#text = test_ds_list[i]['text']
#phonemes, phnid = text2phnid(text, phn2id, 'en')
#phonemes_len_int = len(phonemes)
#phnid_len_int = len(phnid)

#test_dataset_list[0][0]

In [26]:
# The 0th data seemed to have a slower inference time than usual, 
# perhaps because the model took longer to load.
# Therefore, we try to use data from 1 to 101

#eval_jsonl_list[0]

In [27]:
#counter_list = list(range(1,101))
#print(f'counter_list_len: {len(counter_list)}')
#print(f'counter_width: {counter_list[0]}-{counter_list[-1]}')
#print(len(dt_nparr))

In [28]:
# culc for evaluation Indicates
dt_list = [eval_jsonl_list[n]['dt'] for n in range(len(eval_jsonl_list))]
RTF4mel_list = [eval_jsonl_list[n]['RTF4mel'] for n in range(len(eval_jsonl_list))]
utmos_list = [eval_jsonl_list[n]['utmos'] for n in range(len(eval_jsonl_list))]

In [29]:
dt_nparr = np.array(dt_list[1:101])
RTF4mel_nparr = np.array(RTF4mel_list[1:101])
utmos_nparr = np.array(utmos_list[1:101])
print(len(dt_nparr))

100


In [30]:
significant_digits = 5

# for culc difference time to infer text2mel
dt_mean = round_significant_digits(np.mean(dt_nparr), significant_digits=significant_digits)
dt_var = round_significant_digits(np.var(dt_nparr), significant_digits=significant_digits)
dt_std = round_significant_digits(np.std(dt_nparr), significant_digits=significant_digits)
print(f'{target_model}: dt ---------------------------')
print(f'dt mean: {dt_mean}')
print(f'dt var: {dt_var}')
print(f'dt std: {dt_std}')

# for culc RTF4mel to infer text2mel
RTF4mel_mean = round_significant_digits(np.mean(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_var = round_significant_digits(np.var(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_std = round_significant_digits(np.std(RTF4mel_nparr), significant_digits=significant_digits)
print(f'{target_model}: RTF ---------------------------')
print(f'RTF mean: {RTF4mel_mean}')
print(f'RTF var: {RTF4mel_var}')
print(f'RTF std: {RTF4mel_std}')

# for culc utmos to infer 
print(f'{target_model}: utmos ---------------------------')
utmos_mean = round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

gradfreqktts: dt ---------------------------
dt mean: 14.776
dt var: 24.908
dt std: 4.9908
gradfreqktts: RTF ---------------------------
RTF mean: 2.2504
RTF var: 0.0038915
RTF std: 0.062382
gradfreqktts: utmos ---------------------------
utmos mean: 4.1955
utmos var: 0.065222
utmos std: 0.25539
